# Homework III

Diogo Correia (ist199211) & Tomás Esteves (ist199341)

## I. Pen and Paper [12v]

**Consider the problem of learning a regression model from 5 univariate observations ((0.8), (1), (1.2), (1.4), (1.6)) with targets (24,20,10,13,12).**

In [ ]:
import pandas as pd

df1 = pd.read_csv("../data/hw3data.csv")

df1

### 1) [5v] Consider the basis function, $\phi_j(x) = x^j$ , for performing a 3-order polynomial regression,

$$
\hat{z} (x,w) = \sum_{j=0}^3 w_j \phi_j(x) = w_0 + w_1 x + w_2 x^2 + w_3 x^3
$$

Learn the Ridge regression ($l_2$ regularization) on the transformed data space using the closed form solution with $\lambda = 2$.

Hint: use numpy matrix operations (e.g., linalg.pinv for inverse) to validate your calculus

In [ ]:
import numpy.linalg as la
import numpy as np
from numpyarray_to_latex import to_ltx
from numpyarray_to_latex.jupyter import to_jup

In [ ]:
def ltx(array, **wargs):
    print(to_ltx(array, latexarraytype="bmatrix", **wargs))

In [ ]:
df_x = df1.drop("target", axis=1)
target = np.asmatrix(df1["target"]).transpose()

df_x["y0"] = df_x["y1"] ** 0
df_x["y2"] = df_x["y1"] ** 2
df_x["y3"] = df_x["y1"] ** 3

# swap columns
y1 = np.asarray(df_x[["y0", "y1", "y2", "y3"]])
ltx(y1)
to_jup(y1)

In [ ]:
y1_transposed = y1.transpose()
ltx(y1_transposed)
to_jup(y1_transposed)

In [ ]:
y1t_y1 = np.matmul(y1_transposed, y1)
ltx(y1t_y1)
to_jup(y1t_y1)

In [ ]:
y1t_y1_with_lambda = y1t_y1 + 2 * np.identity(4)
ltx(
    y1t_y1_with_lambda,
    mark_color="bblue",
    mark_elements=[
        (0, 0),
        (1, 1),
        (2, 2),
        (3, 3),
    ],
)
to_jup(y1t_y1_with_lambda)

In [ ]:
inverted = np.linalg.inv(y1t_y1_with_lambda)
ltx(inverted)
to_jup(inverted)

In [ ]:
inverted_mult = np.matmul(inverted, y1_transposed)
ltx(inverted_mult)
to_jup(inverted_mult)

In [ ]:
target

In [ ]:
w = np.matmul(inverted_mult, target)
ltx(w, mark_elements=[(i, 0) for i in range(4)])
to_jup(w)

In [ ]:
print("W0: ", w[0])
print("W1: ", w[1])
print("W2: ", w[2])
print("W3: ", w[3])

### 2) [1v] Compute the training RMSE for the learnt regression model.

In [ ]:
def z(x1):
    return np.matmul(w.transpose(), np.array([1, x1, x1**2, x1**3]))


pred = list(map(z, df_x["y1"]))

sum = 0

for i in range(0, 5):
    print(df_x["y1"][i], target[i], pred[i])
    sum += (target[i] - pred[i]) ** 2

rmse = np.sqrt(sum / 5)

rmse

### 3) [6v] Consider a multi-layer perceptron characterized by one hidden layer with 2 nodes. 

Using the activation function $f(x) = e^{0.1x}$ on all units, all weights initialized as 1 (including biases), and the half squared error loss.

Perform one batch gradient descent update (with learning rate $ \eta = 0.1$) for the first three observations (0.8), (1) and (1.2).

> TODO

## Programming and Critical Anlaysis[8v]

**Consider the following three regressors applied on kin8nm.arff data (available at the webpage):**

- linear regression with Ridge regularization term of 0.1
- two MLPs
     - 𝑀𝐿𝑃1 and 𝑀𝐿𝑃2 
- each with two hidden layers of size 10, hyperbolic tangent function as the activation function of all nodes, a maximum of 500 iterations, and a fixed seed (random_state=0). 
- 𝑀𝐿𝑃1 should be parameterized with early stopping while 𝑀𝐿𝑃2 should not consider early stopping. 

Remaining parameters (e.g., loss function, batch size, regularization term, solver) should be set as default

Using a 70-30 training-test split with a fixed seed (random_state=0):

### 4) [4v] **Compute the MAE of the three regressors: linear regression, 𝑀𝐿𝑃1 and 𝑀𝐿𝑃2.**

In [ ]:
from operator import itemgetter
import pandas as pd
from scipy.io.arff import loadarff
from sklearn import feature_selection, model_selection, tree, metrics, preprocessing
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Reading the ARFF file
data = loadarff("../data/kin8nm.arff")
df = pd.DataFrame(data[0])

df.head()

In [ ]:
# Separate features from the outcome (class)
X = df.drop("y", axis=1)
y = df["y"]

y.head()

In [ ]:
# Split the dataset into a training set (70%) and a testing set (30%)
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X.values, y.values, train_size=0.7, random_state=0
)

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
rr = Ridge(alpha=0.01)

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
mlp1 = MLPRegressor(
    hidden_layer_sizes=(10, 10),
    activation="tanh",
    max_iter=500,
    random_state=0,
    early_stopping=True,
)

In [ ]:
mlp2 = MLPRegressor(
    hidden_layer_sizes=(10, 10),
    activation="tanh",
    max_iter=500,
    random_state=0,
    early_stopping=False,
)

In [ ]:
rr.fit(X_train, y_train)
mlp1.fit(X_train, y_train)
mlp2.fit(X_train, y_train)

In [ ]:
rr_pred = rr.predict(X_test)
mlp1_pred = mlp1.predict(X_test)
mlp2_pred = mlp2.predict(X_test)

In [ ]:
print("Ridge Regularization MAE:", metrics.mean_absolute_error(y_test, rr_pred))

In [ ]:
print("MLP1 Regularization MAE:", metrics.mean_absolute_error(y_test, mlp1_pred))

In [ ]:
print("MLP2 Regularization MAE:", metrics.mean_absolute_error(y_test, mlp2_pred))

### 5) [1.5v] **Plot the residues (in absolute value) using two visualizations: boxplots and histograms.**

Hint: consider using boxplot and hist functions from matplotlib.pyplot to this end

In [ ]:
rr_residues = []
mlp1_residues = []
mlp2_residues = []

for i in range(0, len(y_test)):
    rr_residues.append(abs(y_test[i] - rr_pred[i]))
    mlp1_residues.append(abs(y_test[i] - mlp1_pred[i]))
    mlp2_residues.append(abs(y_test[i] - mlp2_pred[i]))

In [ ]:
df = pd.DataFrame({"Ridge": rr_residues, "MLP1": mlp1_residues, "MLP2": mlp2_residues})

df.head()

In [ ]:
sns.boxplot(data=df)

plt.show()

In [ ]:
sns.histplot(data=df)

plt.show()

### 6) [1v] **How many iterations were required for 𝑀𝐿𝑃1 and 𝑀𝐿𝑃2 to converge?**

In [ ]:
print("MLP1 number of iterations:", mlp1.n_iter_)

In [ ]:
print("MLP2 number of iterations:", mlp2.n_iter_)

### 7) [1.5v] **What can be motivating the unexpected differences on the number of iterations?**

**Hypothesize one reason underlying the observed performance differences between the MLPs.**

Read more about the MLP regressor at https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html